In [32]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import time
import json
import os
import shutil
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


#project_path = 'C:\\Users\\C889009\\Downloads\\Web Scraping'
#os.chdir(project_path)


In [33]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

#This is the folder where all the files will be downloaded
# Chromedriver.exe should be placed in this folder

folder_path = "D:\\Office\\AbInBev\\Ops-Analytics\\ABI_Projects\\2021_Projects\\Projects\\MCC\\MCC Recon\\Code\\Web Scrapping\\"
download_path = "D:\\Office\\AbInBev\\Ops-Analytics\\ABI_Projects\\2021_Projects\\Projects\\MCC\\MCC Recon\\"

In [34]:
from selenium import webdriver
import json

settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings),
        'savefile.default_directory': folder_path,
        'download.default_directory' : folder_path}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--kiosk-printing')


### APS Orders by Date Range

In [35]:
#Enter date rage in M/D/YY Format - Example: for First April 2021, enter "4/1/21"

start_date =  '5/1/21'
end_date = '5/10/21'

In [36]:
driver = webdriver.Chrome(executable_path=folder_path +"chromedriver.exe",chrome_options=chrome_options,)

url = "https://www.pbgremit.com/natbill/login.jsp"
#driver = webdriver.Chrome(executable_path="D:/chromedriver.exe")
#driver.set_window_position(-10000,0)
driver.get(url)
u = driver.find_element_by_name('j_username') 
u.send_keys('01-008677')

wait = WebDriverWait(driver, 10)

u = driver.find_element_by_name('j_password')
u.send_keys('PBGPORTAL1')

u = driver.find_element_by_name("Login")
u.click()

time.sleep(3) 


#u = driver.find_element(By.XPATH, '//table/tbody/tr/td/table[1]/tbody/tr/td[3]/a')
u = driver.find_element(By.XPATH, '/html/body/form/table/tbody/tr/td/table/tbody/tr[6]/td/table/tbody/tr/td/table[1]/tbody/tr[3]/td[2]/table[1]/tbody/tr[6]/td[2]/input')
u.click()

time.sleep(2)    

u = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table[1]/tbody/tr/td[4]/a/img')
u.click()


u = driver.find_element_by_name("startDate")
u.send_keys(start_date)
u = driver.find_element_by_name("endDate")
u.send_keys(end_date)

time.sleep(2) 

u = driver.find_element(By.XPATH, '//table/tbody/tr/td/form/div/table/tbody/tr/td[1]/input')
u.click()

time.sleep(2) 

u = driver.find_element(By.XPATH, '//table/tbody/tr/td/form[2]/input[2]')
u.click()

time.sleep(10) 

driver.quit()

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


### APS Orders by invoice Number

In [ ]:
# l = [982982711,196892711,196482711,197413711,351582711,112803711,441772711,834782711,801033711,382862711,978613711,141933711,626282711]
# # l = [197413711]

In [ ]:
l=pd.read_csv("recon_pepsi_underpayment.csv")
l = l[['INV#/APS#']]

In [ ]:
l.head(1)

In [ ]:
All_inv = pd.DataFrame()

driver = webdriver.Chrome(executable_path=folder_path +"chromedriver.exe",chrome_options=chrome_options,)

url = "https://www.pbgremit.com/natbill/login.jsp"
#driver = webdriver.Chrome(executable_path="D:/chromedriver.exe")
#driver.set_window_position(-10000,0)
driver.get(url)
u = driver.find_element_by_name('j_username') 
u.send_keys('01-008677')

wait = WebDriverWait(driver, 10)

u = driver.find_element_by_name('j_password')
u.send_keys('PBGPORTAL1')

time.sleep(1) 


u = driver.find_element(By.XPATH,'//tbody/tr/td/table/tbody/tr[6]/td/table/tbody/tr/td/table[1]/tbody/tr[3]/td[2]/table[1]/tbody/tr[6]/td[2]/input')

u.click()

time.sleep(4) 

for invoice in l["INV#/APS#"]:
    
    u = driver.find_element(By.XPATH, '//table/tbody/tr/td/table[1]/tbody/tr/td[3]/a')
    u.click()

    time.sleep(1)
    
    u = driver.find_element(By.XPATH,"/html/body/table/tbody/tr/td/form/div/table/tbody/tr/td[2]/input[1]")
    u.send_keys(invoice)
    u = driver.find_element(By.XPATH, '//table/tbody/tr/td/form/div/table/tbody/tr/td[2]/input[2]')
    u.click()

    time.sleep(1)
    



    dfs = pd.read_html(driver.page_source)
    items = dfs[11]
    items = items.iloc[1:,:]
    new_header = items.iloc[0] 
    items = items[1:] 
    items.columns = new_header
    items["Invoice_No"] = invoice

    totals = dfs[14]
    totals=totals.iloc[2:,2:].T
    new_header = totals.iloc[0] 
    totals = totals[1:] 
    totals.columns = new_header
    totals["Invoice_No"] = invoice

    full_inv = items.merge(totals,how="left",on="Invoice_No")
    All_inv = All_inv.append(full_inv)
    All_inv = All_inv.reset_index(drop=True)
    time.sleep(1)

    driver.execute_script('window.print();')
    
    time.sleep(1)
    
    filename = folder_path + "Supplier Portal.pdf"
    shutil.move(filename,os.path.join(folder_path,r"{}.pdf".format(invoice)))
    
driver.quit()

# tables=WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.XPATH,"//table/tbody/tr/td/table[3]/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr/td/table")))
# for table in tables:
#     for row in table.find_elements_by_xpath(".//tr"):
#         headers.append(row.find_element_by_xpath('./td').get_attribute('textContent'))
#         body.append(row.find_element_by_xpath('./tr').get_attribute('textContent'))
# driver.execute_script()


In [ ]:
All_inv.shape
All_inv.dropna(inplace=True)
All_inv.shape


In [ ]:
All_inv = All_inv[All_inv['Line#'].apply(lambda x: str(x).isdigit())]

In [ ]:
All_inv.to_excel("APS Orders by Invoice Number.xlsx",index=False)